## importds


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import regularizers, backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.optimizers import AdamW
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score
from joblib import Memory
from sklearn.datasets import load_svmlight_file


## data loader

In [3]:
mem = Memory("./dataset/svm_data", verbose=0)

def dataLoading(path):
    df = pd.read_csv(path)
    y = df['class'].values
    X = df.drop(['class'], axis=1).values
    return X, y

@mem.cache
def get_data_from_svmlight_file(path):
    X_sp, y = load_svmlight_file(path)
    return X_sp.toarray(), y


## loss and call back built for maximizing aucpr

In [ ]:
# this loss tries to pay more attention on samples that are hard to learn
# it uses focal idea on top of deviation so model focuses on big mistakes first
# also it uses some random normal scores as reference and pushes real outliers away
def create_focal_deviation_loss(margin=3.0, gamma=1.1, ref_size=5000):
    ref = K.variable(np.random.normal(size=ref_size), dtype='float32')
    def loss_fn(y_true, y_pred):
        y_true_f = K.cast(y_true, 'float32')
        dev = (y_pred - K.mean(ref)) / (K.std(ref) + K.epsilon())
        in_l = K.abs(dev)
        out_l = K.abs(K.maximum(margin - dev, 0.0))
        base = (1.0 - y_true_f) * in_l + y_true_f * out_l
        weight = K.pow(base / (K.max(base) + K.epsilon()), gamma)
        return K.mean(weight * base)
    return loss_fn

# this callback is very simple it runs after each epoch
# it sees how model does on val data by aupr
# then stores it so training knows if it is good or not
class AUC_Callback(Callback):
    def __init__(self, x_val, y_val):
        super().__init__()
        self.x_val, self.y_val = x_val, y_val

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.x_val, verbose=0).flatten()
        logs['val_aupr'] = average_precision_score(self.y_val, y_pred)


## network

In [ ]:
# this deep network has three big layers then one output
# it uses batch norm, leaky relu and dropout to keep learning stable
# dropout rate and activation slope come from config so we can tune easily
def dev_network_d(input_shape, dropout_rate, activation_slope):
    inp = Input(shape=input_shape)
    x = Dense(1000, kernel_regularizer=regularizers.l2(1e-4))(inp)
    x = BatchNormalization()(x); x = LeakyReLU(alpha=activation_slope)(x); x = Dropout(dropout_rate)(x)
    x = Dense(250, kernel_regularizer=regularizers.l2(1e-4))(x)
    x = BatchNormalization()(x); x = LeakyReLU(alpha=activation_slope)(x); x = Dropout(dropout_rate)(x)
    x = Dense(20, kernel_regularizer=regularizers.l2(1e-4))(x)
    x = BatchNormalization()(x); x = LeakyReLU(alpha=activation_slope)(x); x = Dropout(dropout_rate)(x)
    out = Dense(1, activation='linear')(x)
    return Model(inp, out)

# this function build the full deviation network using focal deviation loss
# it takes config object so we keep code neat and parameters easy to change
def deviation_network(input_shape, cfg):
    model = dev_network_d(input_shape, cfg.dropout_rate, cfg.activation_slope)
    loss = create_focal_deviation_loss(cfg.margin, cfg.gamma)
    optimizer = AdamW(learning_rate=cfg.lr, weight_decay=cfg.weight_decay)
    model.compile(loss=loss, optimizer=optimizer)
    return model


## bacthing

In [ ]:
# this generator makes batches with equal mix of normal and outlier samples
# it picks half batch from outliers with replacement and half from inliers without replacement
# then shuffles and yields the data and labels indicating which are outliers
def batch_generator_sup(x, out_idx, in_idx, batch_size, rng):
    half = max(1, batch_size // 2)
    while True:
        o = rng.choice(out_idx, half, replace=True)
        i = rng.choice(in_idx, batch_size - half, replace=False)
        idx = np.concatenate([i, o])
        rng.shuffle(idx)
        yield x[idx], np.isin(idx, out_idx).astype(np.float32)


## train+test code

In [ ]:
# this function goes through each csv file and train devnet then test it
# it splits data into train, val, test, scales features, drops extra outliers, and adds synthetic noise
# callbacks to track validation aupr, stop early, adjust learning rate, and save best model
# after training it predicts on test set and saves roc and aupr for each dataset
def run_devnet(cfg):
    results = []
    scaler = StandardScaler()
    for fname in os.listdir(cfg.input_path):
        if not fname.endswith('.csv'):
            continue
        name = os.path.splitext(fname)[0]
        X, y = dataLoading(os.path.join(cfg.input_path, fname))
        
        # train/val/test split
        x_tr, x_tmp, y_tr, y_tmp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=cfg.random_seed)
        x_val, x_te, y_val, y_te = train_test_split(x_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=cfg.random_seed)

        # scaling features
        x_tr = scaler.fit_transform(x_tr)
        x_val = scaler.transform(x_val)
        x_te = scaler.transform(x_te)

        # drop extra outliers if more than known limit
        out_idx, in_idx = np.where(y_tr == 1)[0], np.where(y_tr == 0)[0]
        if len(out_idx) > cfg.known_outliers:
            drop = np.random.choice(out_idx, len(out_idx) - cfg.known_outliers, replace=False)
            keep = np.setdiff1d(np.arange(len(y_tr)), drop)
            x_tr, y_tr = x_tr[keep], y_tr[keep]
            out_idx, in_idx = np.where(y_tr == 1)[0], np.where(y_tr == 0)[0]

        # inject synthetic noise by duplicating outliers
        n_noise = int(len(in_idx) * cfg.cont_rate / (1 - cfg.cont_rate))
        synth = x_tr[np.random.choice(out_idx, n_noise, replace=True)]
        x_tr = np.vstack([x_tr, synth])
        y_tr = np.concatenate([y_tr, np.zeros(n_noise)])
        in_idx = np.where(y_tr == 0)[0]

        # build and compile model
        model = deviation_network((x_tr.shape[1],), cfg)
        callbacks = [
            AUC_Callback(x_val, y_val),
            EarlyStopping(monitor='val_aupr', mode='max', patience=2, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_aupr', mode='max', factor=0.5, patience=2, min_lr=1e-6),
            ModelCheckpoint(os.path.join(cfg.model_dir, f"devnet_{name}.keras"), save_best_only=True, monitor='val_aupr', mode='max')
        ]
        steps = max(1, len(in_idx) // cfg.batch_size)
        model.fit(
            batch_generator_sup(x_tr, out_idx, in_idx, cfg.batch_size, np.random),
            steps_per_epoch=steps,
            epochs=20,
            validation_data=(x_val, y_val),
            callbacks=callbacks,
            verbose=0
        )

        # evaluate on test set and collect metrics
        y_score = model.predict(x_te).flatten()
        results.append({
            'dataset': name,
            'roc': roc_auc_score(y_te, y_score),
            'aupr': average_precision_score(y_te, y_score)
        })

    # save results to csv
    pd.DataFrame(results).to_csv('focal_output_final.csv', index=False)
    print("Focal loss results saved to focal_devnet_results.csv")


## call to train+test function

In [ ]:
# choose network size: 1=linear, 2=shallow, 4=deep deviation network
# known_outliers: num outliers to bbe kept
# contam = num outliers
# lr =  learning rate
# weight decay = L2 regularization
# margin = how far outliers must deviate from normal
# gamma = focus opn harder examples (high gamma = more focus to data clsoer to margin)
# act slope ==  leaky relu constraint
if __name__ == "__main__":
    class Config: pass

    cfg = Config()
    cfg.input_path = './dataset/'
    cfg.model_dir = './model'
    cfg.network_depth = 4
    cfg.known_outliers = 30
    cfg.cont_rate = 0.02
    cfg.batch_size = 512
    cfg.lr = 1e-2
    cfg.weight_decay = 1e-5
    cfg.random_seed = 42
    cfg.dropout_rate = 0.3
    cfg.activation_slope = 0.1
    cfg.margin = 3.0
    cfg.gamma = 1.1

    os.makedirs(cfg.model_dir, exist_ok=True)
    run_devnet(cfg)


447/447 [==============================] - 2s 4ms/step
Focal loss results saved to focal_devnet_results.csv
